In [1]:
import pyreadstat
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# === Load SAS file ===
df, meta = pyreadstat.read_sas7bdat('maize-1.sas7bdat')
print("Data shape:", df.shape)
print(df.head())

# === Save to CSV (optional) ===
df.to_csv('maize_data.csv', index=False)
print("Saved to maize_data.csv")


Data shape: (4981, 7393)
   Geno_Code  pop   m1   m2   m3   m4   m5   m6   m7   m8  ...  m7382  m7383  \
0  Z001E0001  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...    0.0    0.0   
1  Z001E0002  1.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  ...    2.0    2.0   
2  Z001E0003  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
3  Z001E0004  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    2.0    2.0   
4  Z001E0005  1.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  ...    0.0    0.0   

   m7384  m7385  m7386  m7387  m7388  m7389  Entry     DtoA  
0    0.0    0.0    0.0    0.0    0.0    0.0    1.0  75.5364  
1    2.0    2.0    2.0    2.0    2.0    2.0    2.0  76.9075  
2    0.0    0.0    0.0    0.0    0.0    0.0    3.0  75.2646  
3    2.0    2.0    2.0    2.0    2.0    2.0    4.0  73.6933  
4    0.0    0.0    0.0    0.0    0.0    0.0    5.0  79.2441  

[5 rows x 7393 columns]
Saved to maize_data.csv


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import time

# === Load data ===
df = pd.read_csv("maize_data.csv")

# === Prepare features and target ===
X = df.drop('DtoA', axis=1)
y = df['DtoA']

# Convert categorical (string) columns to numeric using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# imputer = SimpleImputer(strategy='mean')
# X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# # Standardize data for Ridge and SVR
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import numpy as np

# === Custom RMSE scorer ===
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# === Function to create pipeline ===
def create_pipeline(model):
    return Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

# === Reusable GridSearch function using CV (RMSE only) ===
def run_grid_search(model_pipeline, param_grid, X_train, y_train, X_test, y_test, cv=5):
    """
    Run GridSearchCV with cross-validation on training data, evaluate on test set.
    Only uses RMSE as the scoring metric.
    """
    grid_search = GridSearchCV(
        estimator=model_pipeline,
        param_grid=param_grid,
        scoring=rmse_scorer,
        refit=True,
        cv=cv,
        n_jobs=-1,
        verbose=2
    )

    # Fit GridSearch
    grid_search.fit(X_train, y_train.squeeze())

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # CV metrics (mean of training CV folds)
    cv_results = grid_search.cv_results_
    mean_rmse = np.sqrt(-cv_results['mean_test_score'][grid_search.best_index_])

    # Evaluate on test set
    y_test_pred = best_model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    print("Best params:", best_params)
    print(f"CV RMSE (train): {mean_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")

    return best_model, best_params





### Ridge Regression

In [8]:
# Ridge example
ridge_pipeline = create_pipeline(Ridge())
ridge_param_grid = {'regressor__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

start_time = time.time()
best_ridge, best_params = run_grid_search(
    ridge_pipeline, ridge_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)
elapsed_time = time.time() - start_time

print(f"Time: {elapsed_time:.2f} s")

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best params: {'regressor__alpha': 100.0}
CV RMSE (train): 1.8732
Test RMSE: 3.4956
Time: 156.79 s


### Lasso Regression

In [9]:
# === Lasso example ===
lasso_pipeline = create_pipeline(Lasso(max_iter=5000))
lasso_param_grid = {'regressor__alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}

start_time = time.time()
best_lasso, test_rmse_lasso = run_grid_search(
    lasso_pipeline, lasso_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)
elapsed_time = time.time() - start_time

print(f"Time: {elapsed_time:.2f} s")

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best params: {'regressor__alpha': 0.1}
CV RMSE (train): 1.8533
Test RMSE: 3.3932
Time: 546.18 s


### Elastic Net

In [15]:
# === ElasticNet example ===
elastic_pipeline = create_pipeline(ElasticNet(max_iter=5000))
elastic_param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1.0],
    'regressor__l1_ratio': [0.1, 0.5, 0.7, 0.9]
}

start_time = time.time()
best_elastic, test_rmse_elastic = run_grid_search(
    elastic_pipeline, elastic_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)
elapsed_time = time.time() - start_time

print(f"Time: {elapsed_time:.2f} s")

Fitting 10 folds for each of 16 candidates, totalling 160 fits
Best params: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.5}
CV RMSE (train): 1.8527
Test RMSE: 3.3981


### AIC/BIC

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np
import time

# === Forward Stepwise BIC Selector ===
class ForwardStepwiseBIC(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.best_subset_ = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        remaining_features = list(range(n_features))
        selected_features = []
        best_bic = np.inf

        while remaining_features:
            improved = False
            best_feature = None

            for f in remaining_features:
                trial_features = selected_features + [f]
                X_sub = X[:, trial_features]
                lr = LinearRegression().fit(X_sub, y)
                rss = np.sum((y - lr.predict(X_sub))**2)
                bic = n_samples * np.log(rss / n_samples) + (len(trial_features)+1) * np.log(n_samples)

                if bic < best_bic:
                    best_bic = bic
                    best_feature = f
                    improved = True

            if improved:
                selected_features.append(best_feature)
                remaining_features.remove(best_feature)
            else:
                break  # Stop if no improvement

        self.best_subset_ = selected_features
        return self

    def transform(self, X):
        return X[:, self.best_subset_]

# === Stepwise (BIC) Pipeline ===
stepwise_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('stepwise', ForwardStepwiseBIC()),
    ('regressor', LinearRegression())
])

# No hyperparameters to grid search
stepwise_param_grid = {}

# === Run pipeline with timing ===
start_time = time.time()
best_stepwise, stepwise_metrics = run_grid_search(
    stepwise_pipeline, stepwise_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)
elapsed_time = time.time() - start_time

print(f"# Feats/Comps: {len(best_stepwise.named_steps['stepwise'].best_subset_)}")
print(f"Time: {elapsed_time:.2f} s")


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best params: {}
CV RMSE (train): 1.8636
Test RMSE: 3.4311

=== Stepwise (Forward BIC) Results ===


IndexError: invalid index to scalar variable.

### PCR

In [16]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import time

# === PCR (Principal Components Regression) ===
pcr_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('regressor', LinearRegression())
])

pcr_param_grid = {
    'pca__n_components': [5, 10, 20, 50, 100]  # tune number of components
}

start_time = time.time()
best_pcr, pcr_metrics = run_grid_search(
    pcr_pipeline, pcr_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best params: {'pca__n_components': 50}
CV RMSE (train): 1.8930
Test RMSE: 3.5666


### PLS

In [17]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import time

# === PLS Regression ===
pls_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pls', PLSRegression())
])

pls_param_grid = {
    'pls__n_components': [5, 10, 20, 50, 100]  # tune number of PLS components
}

start_time = time.time()
best_pls, pls_metrics = run_grid_search(
    pls_pipeline, pls_param_grid,
    X_train, y_train,
    X_test, y_test,
    cv=10
)


Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best params: {'pls__n_components': 20}
CV RMSE (train): 1.8715
Test RMSE: 3.4961
